# Text Classification With Argument Level Textual Features

### Importing the required libraries

In [ ]:
from transformers import BertTokenizer, BertForSequenceClassification, Trainer, TrainingArguments
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
import pandas as pd
import spacy

### Reading the dataset

In [ ]:
df = pd.read_csv('compiled_output.csv')

### Removing rows with empty or irrelevant content

In [ ]:
df = df[df['Text'].str.strip().notna()]  # Remove rows with empty strings
df = df[df['Text'].str.strip() != '']  # Remove rows with only whitespace
df = df[~df['Text'].str.contains(r'\[.*\]')]  # Remove rows with non-textual content like '[2]'


### Maping labels to integers

In [ ]:
label_mapping = {
    'background_claim': 0,  # Example: background_claim -> 0
    'own_claim': 1,         # Example: own_claim -> 1
    'data': 2               # Example: data -> 2
}
df['Label'] = df['Label'].map(label_mapping)


### Extract additional required features

In [ ]:
nlp = spacy.load("en_core_web_sm")

def extract_features(text):
    doc = nlp(text)
    tokens = [token.text.lower() for token in doc]
    agreement_count = sum(token in agreement_lexicon for token in tokens)
    disagreement_count = sum(token in disagreement_lexicon for token in tokens)
    hedge_count = sum(token in hedge_words for token in tokens)
    modal_count = sum(token in modal_verbs for token in tokens)
    negation_count = sum(token in negation_lexicon for token in tokens)
    return agreement_count, disagreement_count, hedge_count, modal_count, negation_count

agreement_lexicon = ['agree', 'yes', 'definitely', 'sure', 'absolutely', 'of course', 'agreement', 'consistent', 'consistent with', 'support', 'supports', 'endorses']
disagreement_lexicon = ['disagree', 'no', 'never', 'not', 'don’t', 'won’t', 'disagreement', 'opposes', 'opposed', 'against', 'contradicts']
hedge_words = ['perhaps', 'maybe', 'possibly', 'could', 'might', 'probably', 'likely', 'uncertain', 'should']
modal_verbs = ['can', 'could', 'may', 'might', 'shall', 'should', 'will', 'would']
negation_lexicon = ['not', 'never', 'no', 'none', 'nothing', 'neither']
df[['agreement_count', 'disagreement_count', 'hedge_count', 'modal_count', 'negation_count']] = df['Text'].apply(
    lambda x: pd.Series(extract_features(x))
)

### Spliting the dataset into train and test sets

In [ ]:
train_texts, test_texts, train_labels, test_labels = train_test_split(
    df['Text'].tolist(), df['Label'].tolist(), test_size=0.3, random_state=42
)
train_features, test_features = train_test_split(
    df[['agreement_count', 'disagreement_count', 'hedge_count', 'modal_count', 'negation_count']].values, test_size=0.3, random_state=42
)

### Tokenizing the texts using BERT tokenizer

In [ ]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=128)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=128)


### Combining BERT encodings with additional features

In [ ]:
train_encodings['features'] = train_features.tolist()
test_encodings['features'] = test_features.tolist()

### Converting to Dataset format

In [ ]:
train_dataset = {
    'input_ids': train_encodings['input_ids'],
    'attention_mask': train_encodings['attention_mask'],
    'labels': train_labels,
    'features': train_encodings['features']
}

test_dataset = {
    'input_ids': test_encodings['input_ids'],
    'attention_mask': test_encodings['attention_mask'],
    'labels': test_labels,
    'features': test_encodings['features']
}

# Loading pre-trained BERT model for sequence classification 


In [ ]:
model = BertForSequenceClassification.from_pretrained('bert-base-uncased', num_labels=3)
